In [64]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np
import os, random, math, glob
from IPython.display import Image as IM
from IPython.display import clear_output
from matplotlib import pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [16, 10]
from sklearn.metrics import mean_squared_error
from math import sqrt


In [65]:
train = pd.read_csv('Train (1).csv')
test = pd.read_csv('Test (1).csv')

In [66]:
train.head()

,ward,total_households,total_individuals,target,dw_00,dw_01,dw_02,dw_03,dw_04,dw_05,...,pw_03,pw_04,pw_05,pw_06,pw_07,pw_08,ADM4_PCODE,lat,lon,NL
0,41601001: Ward 1,1674.45058,5888.20750,16.773757,0.933841,0.000846,0.005490,0.000676,0.000000,0.001372,...,0.002848,0.007537,0.000000,0.012928,0,0,ZA4161001,-29.682270,24.734743,0.292039
1,41601002: Ward 2,1736.99230,6735.33812,21.496661,0.696940,0.001253,0.004402,0.000000,0.002301,0.001323,...,0.014566,0.057127,0.019092,0.004131,0,0,ZA4161002,-29.119311,24.757737,3.207775
2,41601003: Ward 3,2403.57591,7273.04995,10.931425,0.810545,0.004517,0.008891,0.003986,0.007735,0.000956,...,0.057560,0.010358,0.001421,0.040881,0,0,ZA4161003,-29.142276,25.094093,0.000000
3,41601004: Ward 4,1740.78737,5734.49046,23.119257,0.659914,0.000000,0.006129,0.000000,0.000813,0.037245,...,0.000000,0.000669,0.000000,0.005011,0,0,ZA4161004,-29.372052,24.942867,2.038778
4,41601005: Ward 5,1730.51451,6657.23835,13.652252,0.950575,0.000655,0.001473,0.000598,0.006999,0.000818,...,0.004859,0.001290,0.000673,0.017629,0,0,ZA4161005,-29.409381,25.290165,0.000000


In [67]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2822 entries, 0 to 2821
Data columns (total 63 columns):
ward                 2822 non-null object
total_households     2822 non-null float64
total_individuals    2822 non-null float64
target               2822 non-null float64
dw_00                2822 non-null float64
dw_01                2822 non-null float64
dw_02                2822 non-null float64
dw_03                2822 non-null float64
dw_04                2822 non-null float64
dw_05                2822 non-null float64
dw_06                2822 non-null float64
dw_07                2822 non-null float64
dw_08                2822 non-null float64
dw_09                2822 non-null float64
dw_10                2822 non-null float64
dw_11                2822 non-null float64
dw_12                2822 non-null int64
dw_13                2822 non-null int64
psa_00               2822 non-null float64
psa_01               2822 non-null float64
psa_02               2822 non-null flo

In [68]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1013 entries, 0 to 1012
Data columns (total 62 columns):
ward                 1013 non-null object
total_households     1013 non-null float64
total_individuals    1013 non-null float64
dw_00                1013 non-null float64
dw_01                1013 non-null float64
dw_02                1013 non-null float64
dw_03                1013 non-null float64
dw_04                1013 non-null float64
dw_05                1013 non-null float64
dw_06                1013 non-null float64
dw_07                1013 non-null float64
dw_08                1013 non-null float64
dw_09                1013 non-null float64
dw_10                1013 non-null float64
dw_11                1013 non-null float64
dw_12                1013 non-null float64
dw_13                1013 non-null float64
psa_00               1013 non-null float64
psa_01               1013 non-null float64
psa_02               1013 non-null float64
psa_03               1013 non-null

In [69]:
train.shape

(2822, 63)

In [70]:
test.shape

(1013, 62)

In [71]:
train.drop(['ADM4_PCODE', 'ward'], axis = 1, inplace = True)
test.drop(['ADM4_PCODE', 'ward'], axis = 1, inplace = True)


In [72]:
y = train['target']
X = train.drop(['target'], axis=1)

In [73]:
import lightgbm as lgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, accuracy_score, roc_auc_score, roc_curve, f1_score
from sklearn.model_selection import GridSearchCV

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=0)
X_train.columns

Index(['total_households', 'total_individuals', 'dw_00', 'dw_01', 'dw_02',
       'dw_03', 'dw_04', 'dw_05', 'dw_06', 'dw_07', 'dw_08', 'dw_09', 'dw_10',
       'dw_11', 'dw_12', 'dw_13', 'psa_00', 'psa_01', 'psa_02', 'psa_03',
       'psa_04', 'stv_00', 'stv_01', 'car_00', 'car_01', 'lln_00', 'lln_01',
       'lan_00', 'lan_01', 'lan_02', 'lan_03', 'lan_04', 'lan_05', 'lan_06',
       'lan_07', 'lan_08', 'lan_09', 'lan_10', 'lan_11', 'lan_12', 'lan_13',
       'lan_14', 'pg_00', 'pg_01', 'pg_02', 'pg_03', 'pg_04', 'lgt_00',
       'pw_00', 'pw_01', 'pw_02', 'pw_03', 'pw_04', 'pw_05', 'pw_06', 'pw_07',
       'pw_08', 'lat', 'lon', 'NL'],
      dtype='object')

In [76]:
estimator = lgb.LGBMRegressor(learning_rate = 0.15, metric = 'l1', 
                        n_estimators = 24, num_leaves = 38)


param_grid = {
    'n_estimators': [x for x in range(24,40,2)],
    'learning_rate': [0.10, 0.125, 0.15, 0.175, 0.2]}
gridsearch = GridSearchCV(estimator, param_grid)

gridsearch.fit(X_train, y_train,
        eval_set = [(X_test, y_test)],
        eval_metric = ['auc', 'binary_logloss'],
        early_stopping_rounds = 5,
              verbose = 1)

[1]	valid_0's binary_logloss: -3.12941	valid_0's auc: 0.771277	valid_0's l1: 7.83817
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.12507	valid_0's auc: 0.673759	valid_0's l1: 7.17636
[3]	valid_0's binary_logloss: -3.12094	valid_0's auc: 0.662234	valid_0's l1: 6.60381
[4]	valid_0's binary_logloss: -3.11644	valid_0's auc: 0.746454	valid_0's l1: 6.07745
[5]	valid_0's binary_logloss: -3.11156	valid_0's auc: 0.73227	valid_0's l1: 5.62969
[6]	valid_0's binary_logloss: -3.10588	valid_0's auc: 0.703901	valid_0's l1: 5.231
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: -3.12941	valid_0's auc: 0.771277	valid_0's l1: 7.83817
[1]	valid_0's binary_logloss: -3.14238	valid_0's auc: 0.470745	valid_0's l1: 7.90232
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.13842	valid_0's auc: 0.458333	valid_0's l1: 7.23041
[3]	valid_0's binary_logloss: -3.13376	valid_0's auc: 0.468085	valid_0's l1: 6.65

[1]	valid_0's binary_logloss: -3.12941	valid_0's auc: 0.771277	valid_0's l1: 7.83817
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.12507	valid_0's auc: 0.673759	valid_0's l1: 7.17636
[3]	valid_0's binary_logloss: -3.12094	valid_0's auc: 0.662234	valid_0's l1: 6.60381
[4]	valid_0's binary_logloss: -3.11644	valid_0's auc: 0.746454	valid_0's l1: 6.07745
[5]	valid_0's binary_logloss: -3.11156	valid_0's auc: 0.73227	valid_0's l1: 5.62969
[6]	valid_0's binary_logloss: -3.10588	valid_0's auc: 0.703901	valid_0's l1: 5.231
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: -3.12941	valid_0's auc: 0.771277	valid_0's l1: 7.83817
[1]	valid_0's binary_logloss: -3.14238	valid_0's auc: 0.470745	valid_0's l1: 7.90232
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.13842	valid_0's auc: 0.458333	valid_0's l1: 7.23041
[3]	valid_0's binary_logloss: -3.13376	valid_0's auc: 0.468085	valid_0's l1: 6.65

[1]	valid_0's binary_logloss: -3.12855	valid_0's auc: 0.771277	valid_0's l1: 7.65225
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.12309	valid_0's auc: 0.779255	valid_0's l1: 6.85848
[3]	valid_0's binary_logloss: -3.11718	valid_0's auc: 0.738475	valid_0's l1: 6.17713
[4]	valid_0's binary_logloss: -3.11108	valid_0's auc: 0.704787	valid_0's l1: 5.60881
[5]	valid_0's binary_logloss: -3.10459	valid_0's auc: 0.664007	valid_0's l1: 5.12563
[6]	valid_0's binary_logloss: -3.09879	valid_0's auc: 0.593972	valid_0's l1: 4.74333
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: -3.12855	valid_0's auc: 0.771277	valid_0's l1: 7.65225
[1]	valid_0's binary_logloss: -3.14123	valid_0's auc: 0.470745	valid_0's l1: 7.72115
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.13523	valid_0's auc: 0.458333	valid_0's l1: 6.9063
[3]	valid_0's binary_logloss: -3.12873	valid_0's auc: 0.542553	valid_0's l1: 6.

[1]	valid_0's binary_logloss: -3.12855	valid_0's auc: 0.771277	valid_0's l1: 7.65225
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.12309	valid_0's auc: 0.779255	valid_0's l1: 6.85848
[3]	valid_0's binary_logloss: -3.11718	valid_0's auc: 0.738475	valid_0's l1: 6.17713
[4]	valid_0's binary_logloss: -3.11108	valid_0's auc: 0.704787	valid_0's l1: 5.60881
[5]	valid_0's binary_logloss: -3.10459	valid_0's auc: 0.664007	valid_0's l1: 5.12563
[6]	valid_0's binary_logloss: -3.09879	valid_0's auc: 0.593972	valid_0's l1: 4.74333
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: -3.12855	valid_0's auc: 0.771277	valid_0's l1: 7.65225
[1]	valid_0's binary_logloss: -3.14123	valid_0's auc: 0.470745	valid_0's l1: 7.72115
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.13523	valid_0's auc: 0.458333	valid_0's l1: 6.9063
[3]	valid_0's binary_logloss: -3.12873	valid_0's auc: 0.542553	valid_0's l1: 6.

[1]	valid_0's binary_logloss: -3.12759	valid_0's auc: 0.771277	valid_0's l1: 7.4674
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.12042	valid_0's auc: 0.772163	valid_0's l1: 6.54788
[3]	valid_0's binary_logloss: -3.11318	valid_0's auc: 0.715426	valid_0's l1: 5.80147
[4]	valid_0's binary_logloss: -3.10592	valid_0's auc: 0.704787	valid_0's l1: 5.19403
[5]	valid_0's binary_logloss: -3.09735	valid_0's auc: 0.753546	valid_0's l1: 4.68254
[6]	valid_0's binary_logloss: -3.08891	valid_0's auc: 0.721631	valid_0's l1: 4.27049
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: -3.12759	valid_0's auc: 0.771277	valid_0's l1: 7.4674
[1]	valid_0's binary_logloss: -3.13999	valid_0's auc: 0.470745	valid_0's l1: 7.54183
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.13291	valid_0's auc: 0.501773	valid_0's l1: 6.64119
[3]	valid_0's binary_logloss: -3.12447	valid_0's auc: 0.56383	valid_0's l1: 5.85

...

[1]	valid_0's binary_logloss: -3.12653	valid_0's auc: 0.771277	valid_0's l1: 7.28259
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.11786	valid_0's auc: 0.859043	valid_0's l1: 6.24611
[3]	valid_0's binary_logloss: -3.10876	valid_0's auc: 0.750887	valid_0's l1: 5.41736
[4]	valid_0's binary_logloss: -3.1001	valid_0's auc: 0.765071	valid_0's l1: 4.76799
[5]	valid_0's binary_logloss: -3.09037	valid_0's auc: 0.726064	valid_0's l1: 4.2691
[6]	valid_0's binary_logloss: -3.0818	valid_0's auc: 0.792553	valid_0's l1: 3.88162
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: -3.12653	valid_0's auc: 0.771277	valid_0's l1: 7.28259
[1]	valid_0's binary_logloss: -3.13866	valid_0's auc: 0.470745	valid_0's l1: 7.3646
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.13005	valid_0's auc: 0.507092	valid_0's l1: 6.35636
[3]	valid_0's binary_logloss: -3.11896	valid_0's auc: 0.41844	valid_0's l1: 5.5446

[1]	valid_0's binary_logloss: -3.12653	valid_0's auc: 0.771277	valid_0's l1: 7.28259
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.11786	valid_0's auc: 0.859043	valid_0's l1: 6.24611
[3]	valid_0's binary_logloss: -3.10876	valid_0's auc: 0.750887	valid_0's l1: 5.41736
[4]	valid_0's binary_logloss: -3.1001	valid_0's auc: 0.765071	valid_0's l1: 4.76799
[5]	valid_0's binary_logloss: -3.09037	valid_0's auc: 0.726064	valid_0's l1: 4.2691
[6]	valid_0's binary_logloss: -3.0818	valid_0's auc: 0.792553	valid_0's l1: 3.88162
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: -3.12653	valid_0's auc: 0.771277	valid_0's l1: 7.28259
[1]	valid_0's binary_logloss: -3.13866	valid_0's auc: 0.470745	valid_0's l1: 7.3646
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.13005	valid_0's auc: 0.507092	valid_0's l1: 6.35636
[3]	valid_0's binary_logloss: -3.11896	valid_0's auc: 0.41844	valid_0's l1: 5.5446

[2]	valid_0's binary_logloss: -3.11492	valid_0's auc: 0.859043	valid_0's l1: 5.96061
[3]	valid_0's binary_logloss: -3.10428	valid_0's auc: 0.804078	valid_0's l1: 5.11855
[4]	valid_0's binary_logloss: -3.09341	valid_0's auc: 0.827128	valid_0's l1: 4.46351
[5]	valid_0's binary_logloss: -3.08291	valid_0's auc: 0.835106	valid_0's l1: 3.97934
[6]	valid_0's binary_logloss: -3.07398	valid_0's auc: 0.762411	valid_0's l1: 3.62937
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: -3.12537	valid_0's auc: 0.771277	valid_0's l1: 7.09863
[1]	valid_0's binary_logloss: -3.13723	valid_0's auc: 0.470745	valid_0's l1: 7.1894
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.12708	valid_0's auc: 0.507092	valid_0's l1: 6.08014
[3]	valid_0's binary_logloss: -3.11416	valid_0's auc: 0.41844	valid_0's l1: 5.2275
[4]	valid_0's binary_logloss: -3.10141	valid_0's auc: 0.427305	valid_0's l1: 4.56728
[5]	valid_0's binary_logloss: -3.08845	valid_0's auc: 0.4

[4]	valid_0's binary_logloss: -3.09341	valid_0's auc: 0.827128	valid_0's l1: 4.46351
[5]	valid_0's binary_logloss: -3.08291	valid_0's auc: 0.835106	valid_0's l1: 3.97934
[6]	valid_0's binary_logloss: -3.07398	valid_0's auc: 0.762411	valid_0's l1: 3.62937
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: -3.12537	valid_0's auc: 0.771277	valid_0's l1: 7.09863
[1]	valid_0's binary_logloss: -3.13723	valid_0's auc: 0.470745	valid_0's l1: 7.1894
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.12708	valid_0's auc: 0.507092	valid_0's l1: 6.08014
[3]	valid_0's binary_logloss: -3.11416	valid_0's auc: 0.41844	valid_0's l1: 5.2275
[4]	valid_0's binary_logloss: -3.10141	valid_0's auc: 0.427305	valid_0's l1: 4.56728
[5]	valid_0's binary_logloss: -3.08845	valid_0's auc: 0.466312	valid_0's l1: 4.06269
[6]	valid_0's binary_logloss: -3.0787	valid_0's auc: 0.413121	valid_0's l1: 3.66351
Early stopping, best iteration is:
[1]	valid_0's binary_lo

GridSearchCV(cv=None, error_score='raise',
       estimator=LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.15, max_depth=-1,
       metric='l1', min_child_samples=20, min_child_weight=0.001,
       min_split_gain=0.0, n_estimators=24, n_jobs=-1, num_leaves=38,
       objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
       silent=True, subsample=1.0, subsample_for_bin=200000,
       subsample_freq=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_estimators': [24, 26, 28, 30, 32, 34, 36, 38], 'learning_rate': [0.1, 0.125, 0.15, 0.175, 0.2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [77]:
print('Best parameters found by grid search are:', gridsearch.best_params_)

Best parameters found by grid search are: {'learning_rate': 0.2, 'n_estimators': 24}


In [78]:
gbm = lgb.LGBMRegressor(learning_rate = 0.2, metric = 'l1', 
                        n_estimators = 24)


gbm.fit(X_train, y_train,
        eval_set=[(X_test, y_test)],
        eval_metric=['auc', 'binary_logloss'],
early_stopping_rounds=5)

[1]	valid_0's binary_logloss: -3.13059	valid_0's auc: 0.800532	valid_0's l1: 7.19913
Training until validation scores don't improve for 5 rounds.
[2]	valid_0's binary_logloss: -3.11882	valid_0's auc: 0.804965	valid_0's l1: 6.08439
[3]	valid_0's binary_logloss: -3.10789	valid_0's auc: 0.680851	valid_0's l1: 5.21651
[4]	valid_0's binary_logloss: -3.09711	valid_0's auc: 0.606383	valid_0's l1: 4.56129
[5]	valid_0's binary_logloss: -3.08637	valid_0's auc: 0.597518	valid_0's l1: 4.06072
[6]	valid_0's binary_logloss: -3.07677	valid_0's auc: 0.560284	valid_0's l1: 3.66195
Early stopping, best iteration is:
[1]	valid_0's binary_logloss: -3.13059	valid_0's auc: 0.800532	valid_0's l1: 7.19913


LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       importance_type='split', learning_rate=0.2, max_depth=-1,
       metric='l1', min_child_samples=20, min_child_weight=0.001,
       min_split_gain=0.0, n_estimators=24, n_jobs=-1, num_leaves=31,
       objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
       silent=True, subsample=1.0, subsample_for_bin=200000,
       subsample_freq=0)

In [79]:
y_pred = gbm.predict(test)
#y_predict = y_pred.astype('int64')

In [80]:
gbm.score(X_train, y_train)

0.3102035537692649

In [81]:
print("RMSE:", sqrt(mean_squared_error(y_train, rf.predict(X_train))))

RMSE: 1.3307687792944694


In [28]:
dsn_test_s = pd.read_csv('Test (1).csv')
submission = pd.DataFrame({'ward':dsn_test_s['ward'], 'target': y_pred})
submission

,ward,target
0,21001001: Ward 1,23.574435
1,21001002: Ward 2,22.065092
2,21001003: Ward 3,25.100745
3,21001004: Ward 4,23.106872
4,21001005: Ward 5,24.017683
...,...,...
1008,19100107: Ward 107,21.115486
1009,19100108: Ward 108,24.017683
1010,19100109: Ward 109,23.574435
1011,19100110: Ward 110,22.065092


In [29]:
filename = 'women_submission.csv'
submission.to_csv(filename,index = False)
print('saved file:' + filename)

saved file:women_submission.csv


In [87]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100) # My go-to quick model for regression tasks.
rf.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [88]:
rf.score(X_train, y_train) # Very good!

0.9834585318009083

In [89]:
y_predict = rf.predict(test)

In [84]:
print("RMSE:", sqrt(mean_squared_error(y_train, rf.predict(X_train))))

RMSE: 1.3098719111036685


In [90]:
dsn_test_s = pd.read_csv('Test (1).csv')
submission = pd.DataFrame({'ward':dsn_test_s['ward'], 'target': y_predict})
submission

,ward,target
0,21001001: Ward 1,18.658697
1,21001002: Ward 2,13.947497
2,21001003: Ward 3,21.683328
3,21001004: Ward 4,16.224131
4,21001005: Ward 5,22.556239
...,...,...
1008,19100107: Ward 107,7.829122
1009,19100108: Ward 108,21.670267
1010,19100109: Ward 109,17.530556
1011,19100110: Ward 110,14.442273


In [92]:
filename = 'women_submission_RandomForest.csv'
submission.to_csv(filename,index = False)
print('saved file:' + filename)

saved file:women_submission_RandomForest.csv
